## Google Dance

In [1]:
#Importing all the libraries required for scraping

from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
# using the crome driver for scraping

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
# Url for main page
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

# updating the page by clicking on search button
browser.find_by_name('btnG').first.click()

# parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

# storing all the data from the main class
Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
# search variables Name and reviews within main class using for loop
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Kolotov Dance Studio'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'Wild Child Dance Studio'}, {'name': 'The Kennedy Dancers'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'bboy bgirl lifestyle'}, {'name': 'Next Step Broadway'}, {'name': 'Performing Arts Workshop'}, {'name': 'Nimbus Dance'}, {'name': 'Salsa Fever On 2 Dance Academy'}, {'name': 'Class Act Performing Arts Studio'}, {'name': 'Dance Palooza Studios'}, {'name': 'My Passion Dance Studio & Entertainment'}, {'name': 'Monroe Street Movement Space'}, {'name': 'Harmony Dance Center'}, {'name': 'Jersey City Ballet'}, {'name': 'Dance Power'}, {'name': 'Union City Music & Dance Academy'}, {'name': 'Garden Street School of the Performing Arts'}, {'name': 'Starlight Dance Center'}, {'name': 'Uptown Dance Studio'}, {'name': 'NJ Salsa Dancing Lessons'}]


In [4]:

Data = []
# Using the geo locator for getting Lat and Long
geolocator = Nominatim(user_agent="my-application")

# getting the link for next pages 

pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

#updating each page link with specific nunmbers to go to next page

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

# parsing pages as the pages updated with above url

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))

# search variables Name, url, adress, phone no, zipcode within main class using for loop

    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
#if the url is empty handling the exception

        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            print('Cant Find the url')

# scraping for phone number

        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        
# if the main directory is empty, handling the exception
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
        
# getting the address
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])

# cleaning the address based on number of delimters

            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                #    print(orgAddress)
                
# Using geolocator to find the lat and long for each address

            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)

# creating a dataframe for Dance
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=3RnnXfG9MZGm_Qb2hJrwBQ&start=20&sa=N&ved=0ahUKEwixoMOT-ZrmAhURU98KHXaCBl4Q8tMDCIAD
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address


In [5]:
#df1.to_excel("outputDance.xlsx")

# creating a dataframe for Dance
df1


,address,location,name,phoneNo,type,url
0,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
1,"357 Broad St %231, Bloomfield, NJ, 07003",NaN,Broadway Performing Arts,9737482787,Dance Classes,http://broadwayperformingarts.net/
2,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]",Just Dancin',,Dance Classes,http://justdancinnj.com/
3,"372 9th St Unit B, Jersey City, NJ, 07302",NaN,Jersey City Muay Thai,,Dance Classes,http://www.jerseycitymuaythai.com/
4,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
5,"355 Eisenhower Pkwy %23100, Livingston, NJ, ...",NaN,Grooves Unlimited Dance Studio,,Dance Classes,http://www.groovesunlimiteddancestudio.com/
6,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
7,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/
8,"1 Pierson Rd, Maplewood, NJ, 07040","[40.724692, -74.279572]",Performers Theatre Workshop,,Dance Classes,http://ptwonline.com/
9,"1550 Lemoine Ave %23204, Fort Lee, NJ, 07024",NaN,"Royal Dance School | Ballet, Hip Hop, K Pop, Y...",,Dance Classes,http://www.royaldanceschoolnj.com/


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

0
[]


In [7]:
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                    #print(orgAddress)
            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



0


In [8]:
#df.to_excel("outputMusic.xlsx")

# Creating a dataframe for Music
df2

""


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

21
[{'name': 'Camp Pinebrook'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New Jersey Kick Boxing'}, {'name': 'New York Sports Clubs'}, {'name': 'The GYM For Kids'}, {'name': 'New York Sports Clubs'}, {'name': 'My Gym Jersey City'}]


In [ ]:
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    #print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                    #print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"]# + dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



1
Cant Find the address


In [ ]:
#df.to_excel("outputSports.xlsx")

# creating a dataframe for sports
df3


## Merged all three dataframe into one DF

In [ ]:
# merging all 3 dataframe for music, Dannce and Sports

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

# Cleaning the dataframet to drop null values for location

df = df.dropna(axis=0, subset=['location','url'])
df = df[df.location != 0]
df

In [ ]:
# exporting to excel

df.to_excel("outputKidsActivities.xlsx") 

In [ ]:
# reseting the index

df=df.reset_index(drop=True)
df.head()

In [ ]:
df = df.rename(columns={'index':'_id'})
df.head()

In [ ]:
df = df.drop(df.index[0])
df

In [ ]:
# adding to directory
split_data = df.to_dict('records')
split_data

In [ ]:
# Ensure that all types are objects
df.dtypes

In [ ]:
# Set up the MongoDB connection through pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# Create DB

db = myclient["KidsRidedb3"]

# Create collection and insert all the data into the MongoDB

mycol = db["activityData21"]
mycol.insert_many(split_data)